# Práctica 1

In [7]:
# Importación de módulos necesarios
using DataFrames
using CSV
using Statistics


## Carga y unificación de datos

In [3]:
# Función recursiva para encontrar todos los CSV
function all_csv_files(dir::String)
    archivos = String[]
    for entry in readdir(dir; join=true)
        if isdir(entry) # si es un directorio, llamar recursivamente
            append!(archivos, all_csv_files(entry)) 
        elseif endswith(entry, ".csv") # si es un archivo CSV
            push!(archivos, entry)
        end
    end
    return archivos
end

function import_dataset()
    base_path = joinpath(pwd(), "Datos_Práctica_Evaluación")
    
    # Obtener todos los archivos CSV recursivamente
    rutas = all_csv_files(base_path)
    println("Archivos CSV encontrados: ", length(rutas))
    
    if isempty(rutas)
        error("No se encontraron archivos CSV en la estructura de carpetas.")
    end
    
    # Leer y combinar todos los CSV
    dfs = DataFrame[]
    for ruta in rutas
        println("Leyendo archivo: ", ruta)
        df_temp = CSV.read(ruta, DataFrame)
        push!(dfs, df_temp)
    end

    # Concatenar todos los DataFrames
    df_unificado = vcat(dfs...)
    println("Dimensiones del DataFrame unificado: ", size(df_unificado))
    return df_unificado
end

# Importar el dataset unificado
df = import_dataset()


Archivos CSV encontrados: 30
Leyendo archivo: /Users/claudiafernandezvilela/Documents/DOCUMENTOS_DEF/IA/tercero/primer_cuatri/MAAAI/practica2/MAAAI/Datos_Práctica_Evaluación/Datos Práctica/Investigador A/day 1/Sujeto_01.csv
Leyendo archivo: /Users/claudiafernandezvilela/Documents/DOCUMENTOS_DEF/IA/tercero/primer_cuatri/MAAAI/practica2/MAAAI/Datos_Práctica_Evaluación/Datos Práctica/Investigador A/day 1/Sujeto_05.csv
Leyendo archivo: /Users/claudiafernandezvilela/Documents/DOCUMENTOS_DEF/IA/tercero/primer_cuatri/MAAAI/practica2/MAAAI/Datos_Práctica_Evaluación/Datos Práctica/Investigador A/day 1/Sujeto_07.csv
Leyendo archivo: /Users/claudiafernandezvilela/Documents/DOCUMENTOS_DEF/IA/tercero/primer_cuatri/MAAAI/practica2/MAAAI/Datos_Práctica_Evaluación/Datos Práctica/Investigador A/day 1/Sujeto_11.csv
Leyendo archivo: /Users/claudiafernandezvilela/Documents/DOCUMENTOS_DEF/IA/tercero/primer_cuatri/MAAAI/practica2/MAAAI/Datos_Práctica_Evaluación/Datos Práctica/Investigador A/day 2/Sujeto_03.

Row,subject,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,tBodyAcc-max()-Y,tBodyAcc-max()-Z,tBodyAcc-min()-X,tBodyAcc-min()-Y,tBodyAcc-min()-Z,tBodyAcc-sma(),tBodyAcc-energy()-X,tBodyAcc-energy()-Y,tBodyAcc-energy()-Z,tBodyAcc-iqr()-X,tBodyAcc-iqr()-Y,tBodyAcc-iqr()-Z,tBodyAcc-entropy()-X,tBodyAcc-entropy()-Y,tBodyAcc-entropy()-Z,"tBodyAcc-arCoeff()-X,1","tBodyAcc-arCoeff()-X,2","tBodyAcc-arCoeff()-X,3","tBodyAcc-arCoeff()-X,4","tBodyAcc-arCoeff()-Y,1","tBodyAcc-arCoeff()-Y,2","tBodyAcc-arCoeff()-Y,3","tBodyAcc-arCoeff()-Y,4","tBodyAcc-arCoeff()-Z,1","tBodyAcc-arCoeff()-Z,2","tBodyAcc-arCoeff()-Z,3","tBodyAcc-arCoeff()-Z,4","tBodyAcc-correlation()-X,Y","tBodyAcc-correlation()-X,Z","tBodyAcc-correlation()-Y,Z",tGravityAcc-mean()-X,tGravityAcc-mean()-Y,tGravityAcc-mean()-Z,tGravityAcc-std()-X,tGravityAcc-std()-Y,tGravityAcc-std()-Z,tGravityAcc-mad()-X,tGravityAcc-mad()-Y,tGravityAcc-mad()-Z,tGravityAcc-max()-X,tGravityAcc-max()-Y,tGravityAcc-max()-Z,tGravityAcc-min()-X,tGravityAcc-min()-Y,tGravityAcc-min()-Z,tGravityAcc-sma(),tGravityAcc-energy()-X,tGravityAcc-energy()-Y,tGravityAcc-energy()-Z,tGravityAcc-iqr()-X,tGravityAcc-iqr()-Y,tGravityAcc-iqr()-Z,tGravityAcc-entropy()-X,tGravityAcc-entropy()-Y,tGravityAcc-entropy()-Z,"tGravityAcc-arCoeff()-X,1","tGravityAcc-arCoeff()-X,2","tGravityAcc-arCoeff()-X,3","tGravityAcc-arCoeff()-X,4","tGravityAcc-arCoeff()-Y,1","tGravityAcc-arCoeff()-Y,2","tGravityAcc-arCoeff()-Y,3","tGravityAcc-arCoeff()-Y,4","tGravityAcc-arCoeff()-Z,1","tGravityAcc-arCoeff()-Z,2","tGravityAcc-arCoeff()-Z,3","tGravityAcc-arCoeff()-Z,4","tGravityAcc-correlation()-X,Y","tGravityAcc-correlation()-X,Z","tGravityAcc-correlation()-Y,Z",tBodyAccJerk-mean()-X,tBodyAccJerk-mean()-Y,tBodyAccJerk-mean()-Z,tBodyAccJerk-std()-X,tBodyAccJerk-std()-Y,tBodyAccJerk-std()-Z,tBodyAccJerk-mad()-X,tBodyAccJerk-mad()-Y,tBodyAccJerk-mad()-Z,tBodyAccJerk-max()-X,tBodyAccJerk-max()-Y,tBodyAccJerk-max()-Z,tBodyAccJerk-min()-X,tBodyAccJerk-min()-Y,tBodyAccJerk-min()-Z,tBodyAccJerk-sma(),tBodyAccJerk-energy()-X,tBodyAccJerk-energy()-Y,tBodyAccJerk-energy()-Z,⋯
,Int64,Float64,Float64,Float64,Float64?,Float64,Float64?,Float64?,Float64?,Float64?,Float64?,Float64,Float64?,Float64,Float64,Float64,Float64?,Float64,Float64,Float64,Float64?,Float64?,Float64,Float64,Float64,Float64?,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64?,Float64,Float64,Float64?,Float64,Float64,Float64,Float64,Float64,Float64,Float64?,Float64?,Float64,Float64,Float64?,Float64?,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64?,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64?,Float64,Float64,Float64?,Float64?,Float64?,Float64?,Float64?,Float64,Float64,Float64,Float64,Float64?,Float64?,Float64?,Float64,⋯
1,1,0.288585,-0.0202942,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,-0.567378,-0.744413,0.852947,0.685845,0.814263,-0.965523,-0.999945,-0.999863,-0.994612,-0.994231,-0.987614,-0.94322,-0.407747,-0.679338,missing,0.929294,-0.853011,0.35991,-0.0585264,0.256892,-0.224848,0.264106,-0.0952456,0.278851,-0.465085,0.491936,-0.190884,0.376314,0.435129,0.66079,0.963396,-0.14084,0.115375,-0.98525,-0.981708,-0.877625,-0.985001,-0.984416,-0.894677,0.892055,-0.161265,0.12466,0.977436,-0.123213,0.0564827,-0.375426,0.899469,-0.970905,-0.97551,-0.984325,-0.988849,-0.917743,-1.0,-1.0,0.113806,-0.590425,0.591146,missing,0.592469,-0.745449,0.720862,-0.712372,0.7113,-0.995112,0.995675,-0.995668,0.991653,0.570222,0.439027,0.986913,0.0779963,0.0050008,-0.0678308,-0.993519,-0.98836,-0.993575,-0.994488,-0.986207,-0.992818,-0.98518,-0.991994,-0.993119,0.989835,0.991957,0.990519,-0.993522,-0.999935,-0.99982,-0.999878,⋯
2,1,0.274262,-0.012

## Visualización

In [4]:
# Número de variables (columnas)
num_variables = ncol(df)
println("\nNúmero de variables: ", num_variables)

# Número de instancias (filas)
num_instancias = nrow(df)
println("Número de instancias: ", num_instancias)

# Número de individuos únicos (columna 'subject')
num_individuos = length(unique(df.subject))
println("Número de individuos: ", num_individuos)

# Columna de salida: la última columna
num_clases_salida = length(unique(df.Activity))
println("Número de clases de salida (Activity): ", num_clases_salida)

clases = unique(df.Activity)
println("Clases de salida: ", clases)



Número de variables: 563
Número de instancias: 10299
Número de individuos: 30
Número de clases de salida (Activity): 6
Clases de salida: String31["STANDING", "SITTING", "LAYING", "WALKING", "WALKING_UPSTAIRS", "WALKING_DOWNSTAIRS"]


## 2. Análisis de valores ausentes

In [26]:
function missing_values_summary(df::DataFrame)
    println("--- Columnas con valores ausentes ---")
    
    # Lista de columnas con missing
    columnas_con_missing = String[]
    
    for col in names(df)
        num_missing = count(ismissing, df[!, col])
        if num_missing > 0
            push!(columnas_con_missing, col)
            pct_missing = num_missing / nrow(df) * 100
            println(col, "': ", round(pct_missing, digits=2), "%")
        end
    end
    
    if isempty(columnas_con_missing)
        println("No hay columnas con valores ausentes.")
    end
    
    # Porcentaje de valores ausentes en todo el dataset
    total_missing = count(ismissing, Iterators.flatten(eachcol(df)))
    total_values = nrow(df) * ncol(df)
    pct_total_missing = total_missing / total_values * 100
    println("Número de columnas numéricas a imputar: ", length(columnas_con_missing))
    println("\nPorcentaje de valores ausentes en todo el dataset: ", round(pct_total_missing, digits=2), "%")
end

# Llamada a la función
missing_values_summary(df)

--- Columnas con valores ausentes ---
tBodyAcc-std()-X': 0.02%
tBodyAcc-std()-Z': 0.02%
tBodyAcc-mad()-X': 0.02%
tBodyAcc-mad()-Y': 0.03%
tBodyAcc-mad()-Z': 0.03%
tBodyAcc-max()-X': 0.02%
tBodyAcc-max()-Z': 0.01%
tBodyAcc-sma()': 0.01%
tBodyAcc-iqr()-X': 0.03%
tBodyAcc-iqr()-Y': 0.01%
tBodyAcc-entropy()-Z': 9.99%
tBodyAcc-arCoeff()-Z,2': 9.99%
tBodyAcc-correlation()-X,Y': 9.99%
tGravityAcc-std()-Y': 9.99%
tGravityAcc-std()-Z': 9.99%
tGravityAcc-mad()-Z': 9.99%
tGravityAcc-max()-X': 9.99%
tGravityAcc-arCoeff()-X,3': 9.99%
tBodyAccJerk-std()-X': 0.01%
tBodyAccJerk-mad()-X': 0.04%
tBodyAccJerk-mad()-Y': 0.04%
tBodyAccJerk-mad()-Z': 0.02%
tBodyAccJerk-max()-X': 0.01%
tBodyAccJerk-max()-Y': 9.99%
tBodyAccJerk-sma()': 0.03%
tBodyAccJerk-energy()-X': 10.0%
tBodyAccJerk-energy()-Y': 0.01%
tBodyAccJerk-iqr()-Y': 0.03%
tBodyAccJerk-iqr()-Z': 0.02%
tBodyAccJerk-entropy()-X': 0.02%
tBodyAccJerk-entropy()-Y': 10.0%
tBodyAccJerk-entropy()-Z': 0.01%
tBodyAccJerk-arCoeff()-X,4': 9.99%
tBodyGyro-mean()

In [30]:
function subject_wise_mean_imputation(df::DataFrame)
    df_imputed = deepcopy(df)
    subject_col = :subject
    
    # Obtener columnas con missing
    num_cols = String[]
    for col in names(df)
        if col != string(subject_col) && any(ismissing, df[!, col])
            push!(num_cols, col)
        end
    end
    
    println("Número de columnas numéricas a imputar: ", length(num_cols))
    
    for subj in unique(df[!, subject_col])
        idx = findall(==(subj), df[!, subject_col])
        subdf = df[idx, :]
        
        for col in num_cols
            # Si todos los valores del sujeto en esta columna son missing
            if all(ismissing, subdf[!, col])
                continue  
            end
            
            # Calcular la media del sujeto en esta columna
            subject_mean = mean(skipmissing(subdf[!, col]))
            
            # Reemplazar missing por la media del sujeto
            df_imputed[idx, col] = coalesce.(df_imputed[idx, col], subject_mean)
        end
    end
    
    return df_imputed
end

# Ejecutar
df_imputed = subject_wise_mean_imputation(df)
missing_values_summary(df_imputed)

Número de columnas numéricas a imputar: 175
--- Columnas con valores ausentes ---
No hay columnas con valores ausentes.
Número de columnas numéricas a imputar: 0

Porcentaje de valores ausentes en todo el dataset: 0.0%
